# 实例1 表的联合查询

In [ ]:
# 导入所需库
import pandas as pd
import numpy as np
from pandas import Series, DataFrame
from datetime import datetime

In [ ]:
# 导入数据文件（excel）
# 6month.xlsx是6月份用户订单数据，7month.xlsx是7月份用户订单数据，注册用户数据.xls是用户信息数据
df_6 = pd.read_excel("6month.xlsx")
df_7 = pd.read_excel("7month.xlsx")
df_users = pd.read_excel(u"注册用户数据.xls")

In [ ]:
# 定义时间类型变量
six_m = pd.to_datetime('2017-06-01')
senven_m = pd.to_datetime('2017-07-01')
eight_m = pd.to_datetime('2017-08-01')

In [ ]:
# 在用户数据表中筛选6月（7月）注册用户
user_6 = df_users[(df_users[u'注册时间'] >= six_m) & (df_users[u'注册时间'] < senven_m)][[u'注册时间',u'药店名称']]
user_7 = df_users[(df_users[u'注册时间'] >= senven_m) & (df_users[u'注册时间'] < eight_m)][[u'注册时间',u'药店名称']]

In [ ]:
# 连接筛选后的表和6月（7月）订单表，找出6月（7月）注册用户的订单
sheet_6 = pd.merge(df_6, user_6, left_on=u'下单用户', right_on=u'药店名称')
sheet_7 = pd.merge(df_7, user_7, left_on=u'下单用户', right_on=u'药店名称')

# 实例2 分组排序并获取每个分组前三条数据

In [ ]:
# 接实例1
# 修改时间数据类型，便于进行时间比较
sheet_6[u'下单时间'] = pd.to_datetime(sheet_6[u'下单时间'])
sheet_7[u'下单时间'] = pd.to_datetime(sheet_7[u'下单时间'])

In [ ]:
def top(df, n=3, column=u'下单时间'):
    return df.sort_values(by=column)[0:n]

In [ ]:
s6 = sheet_6.groupby(u'下单用户').apply(top)

In [ ]:
s7 = sheet_7.groupby(u'下单用户').apply(top)

In [ ]:
# 导出表格到excel文件
writer = pd.ExcelWriter('output.xlsx')
sheet_6.to_excel(writer,u'6月新用户订单')
sheet_7.to_excel(writer,u'7月新用户订单')
s6.to_excel(writer,u'6月新用户前三笔订单')
s7.to_excel(writer,u'7月新用户前三笔订单')
writer.save()

In [ ]:
# 分别获取每个用户前三笔数据

In [ ]:
def get_one(df, n=0, column=u'下单时间'):
    return df.sort_values(by=column)[n:n+1]

In [ ]:
s6_1 = sheet_6.groupby(u'下单用户').apply(get_one, n=0)
s6_2 = sheet_6.groupby(u'下单用户').apply(get_one, n=1)
s6_3 = sheet_6.groupby(u'下单用户').apply(get_one, n=2)

In [ ]:
s7_1 = sheet_7.groupby(u'下单用户').apply(get_one, n=0)
s7_2 = sheet_7.groupby(u'下单用户').apply(get_one, n=1)
s7_3 = sheet_7.groupby(u'下单用户').apply(get_one, n=2)

In [ ]:
writer = pd.ExcelWriter('output.xlsx')
s6_1.to_excel(writer,u'6月新用户第一笔订单')
s6_2.to_excel(writer,u'6月新用户第二笔订单')
s6_3.to_excel(writer,u'6月新用户第三笔订单')
s7_1.to_excel(writer,u'7月新用户第一笔订单')
s7_2.to_excel(writer,u'7月新用户第二笔订单')
s7_3.to_excel(writer,u'7月新用户第三笔订单')
writer.save()

# 实例3 合并拆单数据，并将数据更新到母单

In [ ]:
# 导入数据
df = pd.read_excel("March.xlsx")

In [ ]:
# 去重
df = df.drop_duplicates(u'订单编号')

In [ ]:
# 查看列选项3种方式

# 利用去重函数
df[u'状态'].drop_duplicates()

# 用value_counts也可以实现查看，函数表示各值出现的频率
df[u'状态'].value_counts()

# 用unique查看列选项，不过如果选项是中文会显示编码，不易识别。因而下面用循环打印显示中文
for x in df[u'状态'].unique():
    print x

In [ ]:
# 筛选表
status = [u'已完成', u'已拆单', u'订单审核中']
df1 = df[df[u'状态'].isin(status)]

In [ ]:
# 分别提取母表和子表
df_sub = df1[df1[u'订单编号'].str.len()>20]
df_mo = df1[df1[u'订单编号'].str.len() == 20]

In [ ]:
# 处理字表字段
df_sub[u'订单编号'] = df_sub[u'订单编号'].map(lambda x:x[0:-2])

In [ ]:
# 字表分组求和
df_sub_bak = df_sub.copy() # 备份子单数据，应对新需求
df_sub = df_sub.groupby(u'订单编号')[u'总金额',u'优惠金额'].sum()

In [ ]:
# 根据子单索引（订单编号）修改母单中对应列的值（总金额和优惠金额）
def func_rep1(r):
    # 利用get_loc函数，找到子单订单编号在母单中的索引（get_loc是Index类型的函数，根据值查找对应的索引）
    # 注意异常处理，当查询失败时返回异常（有异常直接跳过）
    try:
        i = pd.Index(df_mo[u'订单编号']).get_loc(r)
    except:
        pass
    else:
        df_mo.loc[i,[u'总金额']] = df_sub.loc[r,[u'总金额']]
        df_mo.loc[i,[u'优惠金额']] = df_sub.loc[r,[u'优惠金额']]
df_sub.index.map(func_rep1)

In [ ]:
# 另外一种思路
# 将母单的索引设置为订单编号，以保持跟子单的类型一致。然后根据索引修改对应值
# inplace参数表示直接修改表，而不是返回一张新表
df_mo.set_index(u'订单编号', inplace=True)

# 函数首先判断订单编号是否在母单索引中。如果在其中则替换相应值
def func_rep1(r):
    if r in df_mo.index:
        df_mo.loc[r,[u'总金额']] = df_sub.loc[r, [u'总金额']]
        df_mo.loc[r,[u'优惠金额']] = df_sub.loc[r, [u'优惠金额']]
df_sub.index.map(func_rep1)

In [ ]:
# 如果要求将子单中存在而母单不存在的数据加入母单，使用下面的函数（对应上面第一种思路）
def func_rep2(r):
    # 利用get_loc函数，找到子单订单编号在母单中的索引（get_loc是Index类型的函数，根据值查找对应的索引）
    # 注意异常处理，当查询失败时返回异常（有异常直接跳过）
    i = -1
    try:
        i = pd.Index(df_mo[u'订单编号']).get_loc(r)
    except:
        # 找到子单数据合并到母单
        tmp = df_sub_bak[df_sub_bak[u'订单编号'] == r]
        df_mo.loc[df_mo.index.max()+1] = tmp.iloc[0]
    else:
        df_mo.loc[i,[u'总金额']] = df_sub.loc[r,[u'总金额']]
        df_mo.loc[i,[u'优惠金额']] = df_sub.loc[r,[u'优惠金额']]
df_sub.index.map(func_rep2)

In [ ]:
# 函数首先判断订单编号是否在母单索引中。如果在其中则替换相应值，否则将整条数据插入母单（对应上面思路2）
df_mo.set_index(u'订单编号', inplace=True)
def func_rep2(r):
    if r in df_mo.index:
        df_mo.loc[r,[u'总金额']] = df_sub.loc[r, [u'总金额']]
        df_mo.loc[r,[u'优惠金额']] = df_sub.loc[r, [u'优惠金额']]
    else:
        df_mo.loc[r] = df_sub_bak[df_sub_bak[u'订单编号'] == r].iloc[0]
df_sub.index.map(func_rep2)

# 实例4 用户聚类分析

* 1.首先读取全年数据
* 2.从原表中提取用户分类因素。本例中有4个，最后使用3个
* 3.使用K-means聚类分析（快速聚类）
* 4.调用pyechars开源库画图

In [ ]:
# 加载相关库

In [ ]:
import pandas as pd
import numpy as np
from pandas import Series, DataFrame
from datetime import datetime

### 读取全年数据

In [ ]:
df1 = pd.read_excel(r"table\1m.xlsx")
df2 = pd.read_excel(r"table\2-5m.xlsx")
df3 = pd.read_excel(r"table\6m.xlsx")
df4 = pd.read_excel(r"table\7m.xlsx")
df5 = pd.read_excel(r"table\8m.xlsx")
df6 = pd.read_excel(r"table\201608.xlsx")
df7 = pd.read_excel(r"table\201609-10.xlsx")
df8 = pd.read_excel(r"table\201611-12.xlsx")

In [ ]:
# 合并多个表数据
df = pd.concat([df1,df2,df3,df4,df5,df6,df7,df8],ignore_index=True)

### 从原表中提取用户分类因素

In [ ]:
# 因素4：独家品种金额

In [ ]:
# 读取条件列表
df9 = pd.read_excel(r"table\dujia.xlsx",sheetname="Sheet2")

# 条件表数据转换为list类型
djsp = list(df9['编号'].values)

In [ ]:
status = [u'已完成', u'配送中', u'订单审核中', u'已送达', u'出库中', u'未支付']
df_d = df[(df[u'状态'].isin(status)) & (df[u'商品编号'].isin(djsp))]

In [ ]:
# 分组求和
c4 = df_d.groupby(u'下单用户')[u'商品总额'].sum()

In [ ]:
# 因素3：累计品规数

In [ ]:
df_s = df[df[u'状态'].isin(status)]

In [ ]:
# 1.依据“下单用户”分组，
# 2.组内先对‘条码’去重
# 3.对去重后的‘条码’数据求计数值，得到每个用户的“累计品规数”

def func(df):
    return df.drop_duplicates(u'条码')[u'条码'].count()
c3 = df_s.groupby(u'下单用户').apply(func)

In [ ]:
# 因素1：用户最近一次的下单时间

In [ ]:
status1 = [u'已完成', u'配送中', u'订单审核中', u'已送达', u'出库中', u'未支付', u'已拆单']
df_t = df[(df[u'状态'].isin(status1)) & (df[u'订单编号'].str.len() == 20)]

In [ ]:
# 将下单时间转换为datetime类型，便于进行时间运算
df_t[u'下单时间'] = pd.to_datetime(df_t[u'下单时间'])

In [ ]:
c1 = df_t.groupby(u'下单用户')[u'下单时间'].max()

In [ ]:
# 因素2：近30天下单频次

In [ ]:
# 去重
df_c= df_t.drop_duplicates(u'订单编号')

In [ ]:
# 定义时间截点（30天前）
# 这里最理想的做法是用当前时间减去30，得到30天前的时间。不过由于时间类型的计算需要将标准时间类型转换成时间戳再计算，这里简单处理
ago_30 = pd.to_datetime('2017-07-16')

In [ ]:
# 先筛选出近30天的订单，然后计算用户下单频次
c2 = df_c[(df_c[u'下单时间']>ago_30)]
c2 = c2.groupby(u'下单用户')[u'订单编号'].count()

In [ ]:
# 合并四个因素的值，创建用户表。注意合并顺序

In [ ]:
# 实际操作是将4个Series类型合并为一张表
# 先将其中一个Series类型转换为DataFrame类型，然后将其余3个Series按列添加到表中
users = DataFrame(c1)
users[u'近30天下单频次'] = c2
users[u'累计品规数'] = c3
users[u'独家品种金额'] = c4

In [ ]:
# 处理无效值
users = users.fillna(0)

In [ ]:
# 依据实际情况，选取3个分类因子。（删除用户最近一次购买时间数据）

In [ ]:
# 最终待分析用户表
users1 = users.loc[:,[u'近30天下单频次',u'累计品规数',u'独家品种金额']]

### 使用K-means聚类分析（快速聚类）

In [ ]:
 # 数据标准化

In [ ]:
# 由于3个因子数据单位不同（有数量，频次，金额），必须对数据进行标准化
# 通常有0-1标准化和z标准化，这里选用z标准化
# Z标准化公式：1.0*(data - data.mean())/data.std()

In [ ]:
t1 = 1.0*(users1[u'近30天下单频次'] - users1[u'近30天下单频次'].mean())/users1[u'近30天下单频次'].std()
t2 = 1.0*(users1[u'累计品规数'] - users1[u'累计品规数'].mean())/users1[u'累计品规数'].std()
t3 = 1.0*(users1[u'独家品种金额'] - users1[u'独家品种金额'].mean())/users1[u'独家品种金额'].std()

In [ ]:
# 创建标准化数值用户表
users_std = users1.copy()
users_std[u'近30天下单频次'] = t1
users_std[u'累计品规数'] = t2
users_std[u'独家品种金额'] = t3

In [ ]:
# 聚类分析
# 数值为连续值，选用k-means聚类分析

In [ ]:
# 指定K值（即分类数）
# clf：分类参数值
# y_pred：分类结果列表

from sklearn.cluster import KMeans
clf = KMeans(n_clusters=3)
y_pred = clf.fit_predict(users_std.values)

In [ ]:
# 将分类结果添加到用户表
users_std[u'分类结果'] = Series(y_pred, index = users_std.index)

### 调用pyechars开源库画图

In [ ]:
# pyechars库可以绘制较为美观的图形，当然也可以用plt画图

In [ ]:
# 因为有三个因子，所以画个三维图。画图时不需要聚类结果数据，提前删除

from pyecharts import Scatter3D

scatter3D = Scatter3D("用户分组", width=1200, height=600)
scatter3D.add("分类0", users_std[users_std[u'分类结果'] == 0].drop(u'分类结果',axis=1).values)
scatter3D.add("分类1", users_std[users_std[u'分类结果'] == 1].drop(u'分类结果',axis=1).values)
scatter3D.add("分类2", users_std[users_std[u'分类结果'] == 2].drop(u'分类结果',axis=1).values)
scatter3D

In [ ]:
# 有时候需要原始数据图像（标准化之前的数据），再画一个原始数据图
users_source = users1.copy()
users_source[u'分类结果'] = Series(y_pred, index = users_source.index)

scatter3D = Scatter3D("用户分组", width=1200, height=600)
scatter3D.add("分类0", users_source[users_source[u'分类结果'] == 0].drop(u'分类结果',axis=1).values)
scatter3D.add("分类1", users_source[users_source[u'分类结果'] == 1].drop(u'分类结果',axis=1).values)
scatter3D.add("分类2", users_source[users_source[u'分类结果'] == 2].drop(u'分类结果',axis=1).values)
scatter3D

In [ ]:
# 调整分类数为4，将用户分成4类。重复前面3类用户过程

In [ ]:
from sklearn.cluster import KMeans

clf = KMeans(n_clusters=4)
y_pred = clf.fit_predict(users_std.drop(u'分类结果',axis=1).values)

In [ ]:
# 更新分类结果
users_std4 = users_std.copy()
users_std4[u'分类结果'] = Series(y_pred, index = users2.index)

In [ ]:
scatter3D = Scatter3D("用户分组", width=1200, height=600)
scatter3D.add("分类0", users_std4[users_std4[u'分类结果'] == 0].drop(u'分类结果',axis=1).values)
scatter3D.add("分类1", users_std4[users_std4[u'分类结果'] == 1].drop(u'分类结果',axis=1).values)
scatter3D.add("分类2", users_std4[users_std4[u'分类结果'] == 2].drop(u'分类结果',axis=1).values)
scatter3D.add("分类3", users_std4[users_std4[u'分类结果'] == 3].drop(u'分类结果',axis=1).values)
scatter3D

In [ ]:
# 导出结果

In [ ]:
writer = pd.ExcelWriter('用户分组.xlsx')
users.to_excel(writer,u'用户表4维度')
users1.to_excel(writer,u'用户表3维度')
users_std.to_excel(writer,u'用户标准化（3维度）数据分组')
users_std4.to_excel(writer,u'用户标准化（4维度）数据分组')
users_source.to_excel(writer,u'用户原数据（3维度）数据分组')
writer.save()

### 查看统计结果

In [ ]:
# 查看分类统计结果（3分类）
users_std[u'分类结果'].value_counts()

In [ ]:
# 查看分类统计结果（4分类）
users_std4[u'分类结果'].value_counts()

In [ ]:
# 另一种办法查看分类统计结果（样本计数）
pd.Series(clf.labels_).value_counts()

In [ ]:
# 查看分类数据均值表（3分类）
users_std.pivot_table(index =[u'分类结果'], values=[u'近30天下单频次',u'累计品规数', u'独家品种金额'], aggfunc=np.mean)

In [ ]:
# 查看分类数据均值表（4分类）
users_std4.pivot_table(index =[u'分类结果'], values=[u'近30天下单频次',u'累计品规数', u'独家品种金额'], aggfunc=np.mean)

In [ ]:
# 如果要查看每个类别的聚类中心，可以按下面操作
pd.DataFrame(clf.cluster_centers_)

In [ ]:
# 附：matplotlib库图像（参考）
# 平面图展示
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
plt.rc('figure', figsize=(10, 6))
pca = PCA()
data =pca.fit_transform(users_std.values)
data = pd.DataFrame(data,index=users_std.index)
d = data[clf.labels_==0]
plt.plot(d[0],d[1],'r.')
d = data[clf.labels_==1]
plt.plot(d[0],d[1],'go')
d = data[clf.labels_==2]
plt.plot(d[0],d[1],'b*')
plt.show()